#### Water Jug Problem

The "water jug" problem can be stated as follows:  
* you have a jug with a capacity of 7 liters of water, and a jug with a capacity of 3 liters of water, 
* you have a water source that will fill either jug to capacity
* you have a drain that can be used to empty either jug  

The available actions are
* fill either jug from the water source
* empty either jug into the drain
* pour some water from a source jug into a destination jug.  The amount actually poured depends on the water level of the source jug and the capacity of the destination jug.   For example
  * if you tried to pour the 3-capacity jug containing 2 liters into the 7-capacity jug containing 6 liters, the 3-capacity jug would contain 1 and the 7-capacity jug would contain 7
  * if you tried to pour the 7-capacity jug containing 3 liters into the 3-capacity jug containing 0 liters, the 3-capacity jug would contain 3 liters and the 7-capacity jug would contain 0 liters
  
It costs 5 to fill either jug regardless of how much water is required.  It costs 3 to empty either jug regardless of how much water is emptied.  It costs 1 to transfer water from one jug to another, regardless of how much water is transferred.

A *problem* consists of 
* Initial -- the initial contents of both jugs
* Goal -- a desired fill level for each jug

So for example, a problem is to find the lowest cost sequence of actions that starts with both jugs empty and results in 0 liters in the 3-liter jug and 3 liters in the 5-liter jug

In [1]:
## First define your world state
##  You should represent your actions as follows:
##     ("fill", "3cap" ) and ("fill", "7cap")
##     ("empty" "3cap") and ("empty", "7cap")
##     ("pour", "3cap", <amount>) and ("pour", "7cap", <amount)  -- where for example
##        ("pour", "3cap")   means "Pour from the 3cap jug to the 7cap jug"
##              The rules above determine how much water is actually poured

from searchClientInterface import WorldState
import copy

class JugWorldState(WorldState):
    def __init__(self, capacities, levels):
        self.capacities = capacities
        self.levels = levels

    def successors(self):
        successors = []
        for i in range(len(self.capacities)):
            for j in range(len(self.capacities)):
                if i != j:
                    # Pour water from jug i to jug j
                    amount_poured = min(self.levels[i], self.capacities[j] - self.levels[j])
                    if amount_poured > 0:  # Check if pouring makes a difference
                        new_levels = self.levels.copy()
                        new_levels[i] -= amount_poured
                        new_levels[j] += amount_poured
                        successors.append((JugWorldState(self.capacities, new_levels), f"Pour Jug{i+1} to Jug{j+1}"))
        
        # Fill or empty each jug individually
        for i in range(len(self.capacities)):
            # Fill jug i
            fill_levels = self.levels.copy()
            fill_levels[i] = self.capacities[i]
            if fill_levels != self.levels:  # Check if filling makes a difference
                successors.append((JugWorldState(self.capacities, fill_levels), f"Fill Jug{i+1}"))

            # Empty jug i
            empty_levels = self.levels.copy()
            empty_levels[i] = 0
            if empty_levels != self.levels:  # Check if emptying makes a difference
                successors.append((JugWorldState(self.capacities, empty_levels), f"Empty Jug{i+1}"))

        return successors


In [2]:
from searchClientInterface import Problem

class JugProblem(Problem):
    def __init__(self, capacities, initials, goals):
        self.capacities = capacities
        self.initials = initials
        self.goals = goals

    def initial(self):
        return JugWorldState(self.capacities, self.initials)

    def isGoal(self, state):
        return state.levels == self.goals

In [6]:
from searchClientInterface import BFSEvaluator, DFSEvaluator
from searchFramework import aStarSearch

# Define the problem
initial = [0, 0]
goal = [0, 5]
problem = JugProblem([3, 7], initial, goal)

# Test A* search with BFSEvaluator
bfs_evaluator = BFSEvaluator()
bfs_result, bfs_stats = aStarSearch(problem, bfs_evaluator)
print("BFS Result:", bfs_result)
print("BFS Stats:", bfs_stats)

# Test A* search with DFSEvaluator
dfs_evaluator = DFSEvaluator()
dfs_result, dfs_stats = aStarSearch(problem, dfs_evaluator, limit=1000)
print("DFS Result:", dfs_result)
print("DFS Stats:", dfs_stats)


BFS Result: ['Fill Jug2', 'Pour Jug2 to Jug1', 'Empty Jug1', 'Pour Jug2 to Jug1', 'Empty Jug1', 'Pour Jug2 to Jug1', 'Fill Jug2', 'Pour Jug2 to Jug1', 'Empty Jug1']
BFS Stats: (23.89469200000002, 10542, 0, 20122)
DFS Result: None
DFS Stats: (0.1692740000000299, 1001, 0, 2499)


Answer these questions (for this problem only):
* Did DFS return the shortest plan?
* In terms of Nodes explored, which method worked better?

Based on the output:

- DFS did not return a plan (`DFS Result: None`), and it explored 1001 nodes.
- BFS returned a plan with actions `['Fill Jug2', 'Pour Jug2 to Jug1', 'Empty Jug1', 'Pour Jug2 to Jug1', 'Empty Jug1', 'Pour Jug2 to Jug1', 'Fill Jug2', 'Pour Jug2 to Jug1', 'Empty Jug1']` and explored 10542 nodes.


1. **Did DFS return the shortest plan?**
   - Since DFS did not return a plan, we cannot directly compare the lengths of plans. However, DFS tends to find a solution faster but may not guarantee the shortest path.

2. **In terms of Nodes explored, which method worked better?**
   - DFS explored fewer nodes (1001) compared to BFS (10542). In this case, DFS was more efficient in terms of nodes explored.

The effectiveness of DFS (in terms of nodes explored) can depend on the nature of the problem and how the state space is organized.